# Treelets Python Implementation

Python implementation of the treelets algorithm to cement my understanding. Working functions will be wrapped in a module and used to compare the treelets decomposition to PCA. 

## Setup

In [210]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import sys
import os

## Functions

In [274]:
def jacobi_rotation(C,a,b):
    """
    Finds a 2x2 Jacobi rotation matrix that decocorrelates two variables. 
    Args: 
        - C: variance-covariance matrix
        - a: index of first variable
        - b: index of second variable
    """
    
    p = len(C)
    C_aa = C[a,a]
    C_bb = C[b,b]
    C_ab = C[a,b]
    
    theta = 0.5*np.arctan(2*C_ab/(C_aa - C_bb))
    cos_theta = np.cos(theta)
    sine_theta = np.sin(theta)
    
    rotation = np.array([[cos_theta, -sine_theta],
                  [sine_theta, cos_theta]])
    
    return rotation

In [275]:
def rotate_covariance_matrix(C, a, b, J):
    """
    De-correlates two variables given their index and a Jacobi rotation matrix. 
    Args: 
        - J: 2x2 Jacobi rotation matrix
        - C: variance-covaraince matrix 
        - a: index of first variable
        - b: index of second variable
    """
    
    p = C.shape[0]
    R = np.identity(p)
    R[a,a], R[a,b], R[b,a], R[b,b] = J.flatten()
    C = np.matmul(np.matmul(R.transpose(),C),R)
    
    return C

In [374]:
def treelet_decomposition1(C, L): 

    treelet = {0:{"C": C, 
                  "J": None,
                  "pair": (None, None), 
                  "order": (None, None)}
              }
    mask = []
    
    for l in range(1,L):
        
        # find most correlated vars 
        which_max = np.abs(np.triu(C, 1))
        if (l > 1):
            which_max[mask,:] = -1
            which_max[:,mask] = -1    
        a, b = np.unravel_index(np.argmax(which_max, axis = None), which_max.shape)
        print((a,b))
        
        J = jacobi_rotation(C, a, b)
        C = rotate_covariance_matrix(C,a,b,J)
        
        treelet[l] = {"C": C,
                      "J": J,
                      "pair": (a,b), 
                      "order": (1,0) if C[a,a] > C[b,b] else (0,1)
                     }
        
        mask += [b if C[a,a] > C[b,b] else a]
        
    return treelet

## Tests 

In [356]:
# load data
dat = pd.read_csv("../data/Ahat.csv")
X = dat.drop("Unnamed: 0", axis = 1)\
       .to_numpy()

In [357]:
Jx = jacobi_rotation(X,10,13)

In [358]:
jj = np.array([[0.748697326664324,-0.662911994947816],
               [0.662911994947816, 0.748697326664324]])

In [359]:
rotate_covariance_matrix(X,10,13,Jx)[10,10]

1.0777816030057419

In [379]:
tree = treelet_decomposition1(X,10)

(10, 13)
(42, 43)
(22, 24)
(10, 14)
(10, 24)
(10, 25)
(10, 11)
(10, 18)
(31, 32)


In [389]:
tree[4]["J"]

array([[ 0.8528102 , -0.52222099],
       [ 0.52222099,  0.8528102 ]])

In [391]:
pd.read_csv("../data/clean UK macro data.csv").to_numpy()

array([[ 5.60225555e-01, -9.25000000e-05, -6.35917900e-03, ...,
         1.57928130e-02, -2.48866630e-02,  1.14245700e-02],
       [ 0.00000000e+00, -9.07000000e-05,  1.55628200e-03, ...,
         1.40282070e-02,  1.57928130e-02, -2.48866630e-02],
       [ 9.72905520e-01, -3.20913100e-03, -2.20000000e-05, ...,
         3.46132400e-03,  1.40282070e-02,  1.57928130e-02],
       ...,
       [ 5.62853268e-01, -3.94783300e-03, -4.81945600e-03, ...,
         1.66089100e-03,  3.86237800e-03, -1.45741000e-04],
       [-1.87265972e-01, -1.95477200e-03,  5.58589100e-03, ...,
         9.84632200e-03,  1.66089100e-03,  3.86237800e-03],
       [ 1.02565002e+00,  4.76304700e-03, -5.68111800e-03, ...,
         1.89921400e-03,  9.84632200e-03,  1.66089100e-03]])